Load in datasets and add them to a graph. Add all triples to the knowledge graph as the these files reflect partitions from experiments in [COMET-ATOMIC 2020: On Symbolic and Neural Commonsense Knowledge Graphs](https://arxiv.org/pdf/2010.05953.pdf) (Hwang et al., 2021), however we are interested in the full ATOMIC 2020 knowledge base without partitions for now.

In [1]:
import csv
from collections import defaultdict

heads = defaultdict(list) # a dictionary with a head (string) as the key and a list of (relation, tail) tuples as the value (graph adjacency list)
tails = defaultdict(list) # same as above, but w/ tail as the key and a list of (relation, head) tuples as the value; all edge directions are thus reversed

train_file = open('data/train.tsv')
train = csv.reader(train_file, delimiter='\t')
for row in train:
    heads[row[0]].append((row[1], row[2]))
    tails[row[2]].append((row[1] + '⁻¹', row[0]))

dev_file = open('data/dev.tsv')
dev = csv.reader(dev_file, delimiter='\t')
for row in dev:
    heads[row[0]].append((row[1], row[2]))
    tails[row[2]].append((row[1] + '⁻¹', row[0]))

test_file = open('data/test.tsv')
test = csv.reader(test_file, delimiter='\t')
for row in dev:
    heads[row[0]].append((row[1], row[2]))
    tails[row[2]].append((row[1] + '⁻¹', row[0]))



We'll use a graph structure based on adjacency lists since I expect any knowledge base to be a relatively sparse graph (most nodes will only have edges to a small fraction of total nodes)

Some notes about the knowledge base: ATOMIC 2020 represents edges as unidirectional from the head to the tail. For our convencience, and because it could be useful for tasks, we give each node two adjacency lists. The heads adjacency list has edges in the direction that they are in the knowledge base, and the tails adjacency list has edges in the reversed direction for convenient accessing/lookups.

Though symmetric, reciprocal relationships are possible, this should not be as common as ATOMIC 2020 (unlike knowledge bases like ConceptNet) deliberately contains more information about cause-effect, if-then information. It is more focused on inferential knowledge than taxonomic knowledge. Let's print out a selection of five triplets from ATOMIC 2020 to show this.

In [2]:
import random

triples_list = list(heads.items())
len_triples = len(triples_list)
for i in range(5):
    random_index = random.randrange(len_triples)
    selection = triples_list[random_index]
    head = selection[0]
    relationship = selection[1][0][0]
    tail = selection[1][0][1]
    print(head + ' ' + relationship + ' ' + tail) # print the head and just one predicate (relationship + tail)


PersonX knows PersonY's stuff oEffect get closer to PersonX
PersonX seeks PersonY explanation oEffect they tell personX
PersonX makes it home in time oEffect none
PersonX increases the ___ by increasing oEffect none
door key ObjectUse unlock the door


For our shortest path function, we'll use a simple BFS search. There are no weights for ATOMIC 2020 allowing us to use this simple solution. 

If a more performant solution were needed, it would be interesting seeing if heuristics from using word embeddings + maybe a little depth would increase average performance. Although in the aforementioned paper about COMET-ATOMIC, (as far as I understand; need to ask) fast searches are not required as the knowledge bases are used to tune pretrained LM's. Computation time is therefore not noticeably increased when making a determination of a response to a question when the model is deployed.

In [3]:
from collections import deque

def find_shortest_path(starting_node, final_node):
    nodes_to_visit = deque([starting_node])
    paths_for_nodes = deque([starting_node]) # string with text of nodes previously visited
    nodes_visited = set() # for cycle detection
    nodes_to_visit_set = set() # need to not append nodes multiple times; otherwise runtime explodes
    while nodes_to_visit:
        current_node = nodes_to_visit[0]
        normal_edge_connections = heads[current_node]
        reversed_edge_connections = tails[current_node]
        connections_to_current_node = normal_edge_connections + reversed_edge_connections
        for connection in connections_to_current_node:
            relationship, connected_node = connection
            if connected_node == final_node:
                if '⁻¹' not in relationship: # regular direction edge
                    return paths_for_nodes[0] + ' --> ' + relationship + ' ' + connected_node # found final node
                else: # reverse direction edge
                    return paths_for_nodes[0] + ' ' + relationship + ' <-- ' + connected_node # found final node
            elif connected_node not in nodes_visited and connected_node not in nodes_to_visit_set:
                nodes_to_visit.append(connected_node)
                nodes_to_visit_set.add(connected_node)
                if '⁻¹' not in relationship: # regular direction edge
                    paths_for_nodes.append(paths_for_nodes[0] + ' ' + relationship + ' --> ' + connected_node)
                else: # reverse direction edge
                    paths_for_nodes.append(paths_for_nodes[0] + ' <-- ' + relationship + ' ' + connected_node)

        nodes_to_visit.popleft()
        paths_for_nodes.popleft()
        nodes_visited.add(current_node)
    return 'No such path exists between ' + starting_node + ' and ' + final_node

# there is no path
print(find_shortest_path('PersonX washes everything', 'gibberish sfdjkhlghslkdfg'))
print(find_shortest_path('gibberish sfkjhafadf', 'gibberish sfdjkhlghslkdfg'))

# note that no weights with non-randomized BFS can lead to "catch-all" nodes getting high traffic; this may signify a more tenuous connection
# for both of these the 'person' node plays a key factor in a short connection
print(find_shortest_path('happiness', 'go to dealership'))
print(find_shortest_path('cook curry', 'give up'))

# direct connection
print(find_shortest_path('dog', 'food'))

# some more examples
print(find_shortest_path('PersonX washes everything', 'they ran out of soap'))
print(find_shortest_path('happiness', 'love'))
print(find_shortest_path('love', 'dog'))

No such path exists between PersonX washes everything and gibberish sfdjkhlghslkdfg
No such path exists between gibberish sfkjhafadf and gibberish sfdjkhlghslkdfg
happiness <-- NotDesires⁻¹ person Desires --> buy car --> xNeed go to dealership
cook curry HasSubEvent --> cook food <-- CapableOf⁻¹ person --> NotDesires give up
dog --> Desires food
PersonX washes everything isAfter --> PersonX cleans the dishes --> HinderedBy they ran out of soap
happiness <-- xIntent⁻¹ PersonX is having a great time --> xReact love
love ObjectUse⁻¹ <-- dog


Let's now try to use this graph on some examples in examples.txt file. First, we'll do the clerical work of getting our data in order from the text file. (I noticed parsing the file wasn't a requirement after implementing it)

In [4]:
import re

class Example:

    def __init__(self, context, question, choices, correct_answers):
            self.context = context # no context is represented as None
            self.question = question
            self.choices = choices # array of choices
            self.correct_answers = correct_answers # array of correct answers

# set up our examples array using the .txt file
def setup_examples():
    examples = []
    examples_file = open('data/examples.txt')
    lines = examples_file.readlines()
    line_number = 0
    context = None
    while line_number < len(lines):
        current_line = lines[line_number]

        if 'Context:' in current_line:
            context_index = current_line.index(' ') + 1
            context = current_line[context_index:].rstrip()

        if 'Question:' in current_line:
            question_index = current_line.index(' ') + 1
            question = current_line[question_index:].rstrip()
            choices_line = line_number + 3
            choices = []
            while re.search('([a-z])', lines[choices_line][0:3]):
                choices.append((lines[choices_line][4:].rstrip()))
                choices_line += 1
            correct_answer_line = lines[choices_line+1]
            correct_answers = [letter.strip() for letter in re.findall(' [a-z]', correct_answer_line)]
            examples.append(Example(context, question, choices, correct_answers))
            context = None

        line_number += 1

    return examples

examples = setup_examples()

It's important to be able to find relevant nodes in our knowledge graph that represent concepts or tokens from our sentences, yet may have lexical variation, inflections, or particular names that we all want to map back to abstract concepts. We'll use spaCy, a library with models that are pre-trained for various NLP tasks such as part-of-speech tagging and lemmatization.

First, let's convert mentions names and people to 'PersonX' and 'PersonY', which is what ATOMIC 2020 uses.

In [5]:
import spacy
nlp = spacy.load('en_core_web_sm') # the default spaCy pipeline; see https://spacy.io/usage/spacy-101 for details

def replace_people(sentence):
    doc = nlp(sentence)
    person_x_name = ''
    for ent in doc.ents:
            if ent.label_ == 'PERSON':
                person_replacement = 'PersonX' if (person_x_name == ent.text or person_x_name == '') else 'PersonY' # see note below
                sentence = sentence[:sentence.index(ent.text)] + person_replacement + sentence[sentence.index(ent.text) + len(ent.text):] # replace entities left-to-right
                person_x_name = ent.text
    if person_x_name != '': # just print cases where we made an actual substitution
        print(sentence)
    return sentence

for example in examples:

    if example.context is not None: # is there context?
        example.context = replace_people(example.context)
    
    example.question = replace_people(example.question)
    
    for i, choice in enumerate(example.choices):
        example.choices[i] = replace_people(choice)

Visiting New York City interested PersonX but not PersonY because he hates being around crowds of people.
PersonX
PersonX got his first credit card in college. He didn’t have a job so he bought everything on his card. After he graduated he amounted a $10,000 debt. PersonX realized that he was foolish to spend so much money.
PersonX decided to open another credit card.
PersonX decided to devise a plan for repayment.
Growing up on a farm near St. Paul, L. PersonX didn't dream of becoming a judge.
What did PersonX do right after he found out that he became a judge?


The sentences where names were changed to PersonX/PersonY are printed above.

We notice that spaCy missed some names such as Alex and Skylar. I'm using the small English model, and I wonder if the large model would recognize Skylar, a slightly more rare name, for example.

I would like to differentiate between PersonX and PersonY, but it's also noticeable that we have a tricky problem here. I used a simple heuristic (that I actually suspect is fairly reliable except in cases of nicknames or epithets) such that if we've already seen one name and a second name is appears and is different, make it PersonY. But to improve the sentences futher pronouns present a problem. 

To substitute for pronouns, we need coreference resolution. I'm unsure about whether ATOMIC 2020 aimed to make all possevsive pronouns refer to "PersonX" and "PersonY" because looking at the data many occurences of "his" and "her" remain. I looked for libraries that do coreference resolution in Python as state of the art resolution is complex (see the Windograd Schema Challenge) and use neural nets. They learn things like gender association, plurality, etc. in an unsupervised manner not to mention common sense pronoun resolution based on context. [There is currently ongoing effort to integrate this natively into spaCy](https://github.com/explosion/spaCy/pull/7264), but it's not yet available. A workaround of using a development version [doesn't work out of the box for Apple silicon](https://github.com/msg-systems/coreferee/issues/23). Due to this, I'll look at extracting smaller fragments and making other changes.

It's important to note how the knowledge base uses inflections and what it uses as a lemma. We'll also get the lemma of verbs (the infinitive form), which is how ATOMIC 2020 captures most verbs. For example, 'buy food xNeed have money'. We want to be able to match the 'buy food' node if we for example had a question "Paloma bought food for everyone". 

However, in sentences that have PersonX and/or PersonY as a subject the verb is inflected. This means that we should use spaCy to selectively transform our examples.

In [6]:
import mlconjug3 # see https://mlconjug3.readthedocs.io/en/latest/modules.html; note: not great documentation

default_conjugator = mlconjug3.Conjugator(language='en') # library to conjugate verb to 3rd person singular present to match ATOMIC 2020 (ignore rarer 3rd person plural for now)

def replace_verb(sentence):
    doc = nlp(sentence)
    current_idx = 0
    for tok in doc:
        if tok.pos_ == 'VERB' and tok.dep_ == 'ROOT':
            verb_third_person_present = default_conjugator.conjugate(tok.lemma_).conjug_info['indicative']['indicative present']['3s']
            sentence = sentence[:sentence.index(tok.text, current_idx)] + verb_third_person_present + sentence[sentence.index(tok.text, current_idx) + len(tok.text):] # replace verb with present tense
            current_idx = tok.idx
        elif tok.pos_ == 'VERB':
            sentence = sentence[:sentence.index(tok.text, current_idx)] + tok.lemma_ + sentence[sentence.index(tok.text, current_idx) + len(tok.text):] # replace all other verb forms with their lemma
            current_idx = tok.idx
    return sentence
    
for example in examples:

    if example.context is not None: # is there context?
        example.context = replace_verb(example.context)
        print(example.context)
        
    example.question = replace_verb(example.question)
    print(example.question)

    for i, choice in enumerate(example.choices):
        example.choices[i] = replace_verb(choice)

The only baggage the woman check was a drawstre bag, where was she head with it?
The host cancels the party.
What was the cause?
The man uncovees incriminate evidence against his enemy.
What happens as a result?
Alex has a party at his house while his parents were out of town even though they tell him not to.
What will happens to Alex's parents?
Skylar get a letter in the mail, it was from harvard, he was excited.
What will Skylar wants to do next?
They eases the pipe onto the giant pile so it wouldn't burst as it was rather rigid.
What was rather rigid?
visits New York City interested PersonX but not PersonY because he hate being around crowds of people.
What hates being around crowds of people?
PersonX gets his first credit card in college. He didn’t has a job so he buy everything on his card. After he graduate he amounts a $10,000 debt. PersonX realizes that he was foolish to spend so much money.
What is the next sentence?
grow up on a farm near St. Paul, L. PersonX didn't dream of 

These results are not expected to be gramatically correct, but even so we can see in the above printed lines that spaCy is imperfect for lemmatization using it its primitive, built-in tags (missing verbs in independant clauses after commas, misclassifying the gerund "Visiting New York City" as the sentence's verb, etc.) with its classifications, and the ATOMIC dataself itself doesn't lend to the task (There is no lemmatization of the sentence's root verb but lemmatization of other verbs; I would guess this is because it is intended for deep learning tasks that can pick up on these associations w/o the effort involved in coding solutions like this that often have many, many edge cases)

It seems like ATOMIC 2020's use of natural language for nodes (heads and tails) precludes non-neural solutions (or at least they are certainly HinderedBy it)

Given this, let's focus on small samples based on parts of speech (just taking subjects, verbs, and verb + direct objects) and see what the knowledge graph produces.

In [7]:
def extract_tokens(sentence):
    tokens = []
    doc = nlp(sentence)
    for tok in doc:
        if tok.dep_ == 'nsubj':
            tokens.append(tok.text)
        elif tok.dep_ == 'VERB':
            tokens.append(tok.text)
        elif tok.dep_ == 'dobj':
            tokens.append(tok.head.text + ' ' + tok.text) # inspired by the 'catch flu' example, let's search for the direct object after the verb
    return tokens

for index, example in enumerate(examples):

    context_question_tokens = []
    if example.context is not None: # is there context?
        context_tokens = extract_tokens(example.context)
        context_question_tokens += context_tokens
    question_tokens = extract_tokens(example.question)
    context_question_tokens += question_tokens

    choice_tokens = []
    for choice in example.choices:
        choice_tokens = extract_tokens(choice)

    # now cross the context/question tokens with the choice tokens and see if there are any connections
    final_results = []
    for cq_token in context_question_tokens:
        for choice_token in choice_tokens:
            if cq_token != choice_token: # disqualify trivial connection back to self
                potential_path = find_shortest_path(cq_token, choice_token)
                if potential_path[0:7] != 'No such': # filter out negatives
                    print('From example #' + str(index+1) + ': ' + potential_path) # print it as it's a real path



From example #4: parents <-- xEffect⁻¹ PersonX brings children into the world oReact --> happy <-- xReact⁻¹ PersonX eats nachos --> xWant clean mess
From example #4: they <-- xEffect⁻¹ PersonX purchases this book without a cover oEffect --> none <-- oEffect⁻¹ PersonX eats nachos --> xWant clean mess
From example #4: tell him <-- xWant⁻¹ PersonX loves PersonX's husband oEffect --> none <-- oEffect⁻¹ PersonX eats nachos --> xWant clean mess
From example #8: PersonX <-- HinderedBy⁻¹ PersonX brings PersonY back oEffect --> none <-- oEffect⁻¹ PersonX takes PersonY's revenge --> xNeed devise plan
From example #8: PersonX <-- HinderedBy⁻¹ PersonX brings PersonY back oEffect --> none <-- oEffect⁻¹ PersonX takes PersonY's revenge --> xNeed devise plan
From example #8: spend money xIntent --> give gift xNeed --> generous <-- xAttr⁻¹ PersonX brings PersonY back --> HinderedBy PersonX
From example #8: spend money <-- xEffect⁻¹ PersonX always loved ___ oEffect --> none <-- oEffect⁻¹ PersonX takes P

Analysis of results/conclusions: Looking at the above results, extracting tokens from spaCy didn't prove to be fruitful as only 2 out of 9 examples had results from the knowledge graph. Instead, I think using spaCy and an algorithm for edit distance (experimenting with the threshold) and thus using longer substrings could have generated more and better examples. Also, longer substrings are more attractive in terms of capturing semantic meaning.

I think the most salient observation is that 6 out of the 9 examples have a trivial node that connects them; the "none" node. I noted in a comment in the cell that implements the path search algorithm that there can be knowledge nodes (in that case, "person") which have a lot of traffic and thus I believe could represent less meaningful connections or connections that should be given less weight. "none" is an extreme example of this likely does not help with common sense reasoning. A neural net could learn to ignore connections through "none" with enough training, but coding this manually, it is a case that needs to be observed and somehow penalized.

I suspect that generally the most accurate knowledge comes from the shortest paths, as we see in the first path printed that there is a connection  where individual connections make sense, but it suggests the wrong answer. This path may already have many connections and it's probability of being useful shrinks. The other two examples that don't involve none aren't as clearly semantically related to their examples.

In regards to the most common type of knowledge, Atomic 2020 deliberately tries to capture if-then and inferential knowledge. I noticed as well while working with the KB that it has a lot of information on emotions/sentiments. And of course, many events. I didn't hit those due to extraction and matching, but they are there.

Another key conclusion is that I feel that neural net solutions are easier for knowledge graphs that use natural language as using other methods results in a lot of effort to fix edge case behaviours and detect anomalies.